In [ ]:
# configuration cell

epochs = 30

activationFunctions = {
    'conv1':'ReLU',
    'conv2':'ReLU',
    'conv3':'ReLU',
    'conv4':'ReLU',
    'conv5':'ReLU',
    'fc1':'ReLU'
}

list_kernelSize= [3,3,3,3,3]
kernelNumber = [32,32,64,64,128]
listDropout = [0,0,0.5]
nodesfc1 = 1024
classes = 10
learningRate = 0.0001
lr_schedule = 1 # per 10 epochs half the learningRate

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
import numpy as np
import shutil
import os
import wandb
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import RandomCrop, RandomResizedCrop, RandomHorizontalFlip, Resize, CenterCrop, ToTensor, Normalize, Compose
# from util import*

In [ ]:
################################## util.py##############################################################


## dataloader
def loader(t1data,valdata,t2data,batch):
    bs=batch
    bool=True
    bool2=False
    allLoaders = {
        'train' : torch.utils.data.DataLoader(t1data, batch_size=bs,num_workers = 4, shuffle=bool) ,
        'valid' : torch.utils.data.DataLoader(valdata, batch_size=bs,num_workers = 4,  shuffle=bool) ,
        'test'  : torch.utils.data.DataLoader(t2data, batch_size=bs,num_workers = 4,  shuffle=bool2)
    }
    return allLoaders


## transforms to match realModel input dims
def transform():
    
    valResize = 256 #134 #36
    sizeChange = 224 #128#32
    valCenterCrop = sizeChange
    
    #t1_t = Compose([RandomResizedCrop(sizeChange),
    string ='Normalize'
    t1_t = Compose([RandomResizedCrop(sizeChange),
                       RandomHorizontalFlip(),
                       ToTensor(),
                       Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])])
    val_t = Compose([Resize(valResize),
                       CenterCrop(valCenterCrop),
                       ToTensor(),
                       Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])])
    t2_t = Compose([Resize((sizeChange,sizeChange)), 
                      ToTensor(), 
                      Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])])
    
    transforms = {
        'training':   t1_t,
        'validation': val_t,
        'test': t2_t
    }
    
    return transforms

## Load dataset fn
def data_load():
    transforms=transform()
    t1set  = torchvision.datasets.ImageFolder('/kaggle/input/dlasssssss2/inaturalist_12K/train', transforms['training'])
    train, val = random_split(t1set, [8000, 1999])
    t2set   = torchvision.datasets.ImageFolder('/kaggle/input/dlasssssss2/inaturalist_12K/val', transforms['test'])
    return train, val, t2set